In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# bertscore = load("bertscore") 
tokenizer =  AutoTokenizer.from_pretrained('facebook/bart-base',cahe_dir='/ssd_scratch/cvit/aparna/bart')
model = AutoModelForSeq2SeqLM.from_pretrained("/ssd_scratch/cvit/aparna/mbart_simplification_final")

/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_labels)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

In [3]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*10)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*10)
    eval_arg = [inpu,cands,lab]
    if i%100==0:
        metric = metrics_func(eval_arg)
        print(metric)


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  0%|          | 1/10000 [00:01<3:51:10,  1.39s/it]

10 10 10
3
(25.926559358625884, 40.81037555988784)


  1%|          | 100/10000 [01:47<4:43:40,  1.72s/it]

1010 1010 1010
3


  1%|          | 101/10000 [01:49<4:52:12,  1.77s/it]

...

  4%|▍         | 400/10000 [12:19<10:34:14,  3.96s/it]

4010 4010 4010
3


  4%|▍         | 401/10000 [12:29<15:41:00,  5.88s/it]

(36.15792048408291, 44.85257431335065)


  4%|▍         | 435/10000 [14:05<5:40:07,  2.13s/it] 

In [ ]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*1)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*1)
    eval_arg = [inpu,cands,lab]
    if i%100==0:
        metric = metrics_func(eval_arg)
        print(metric)


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)
